In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/processed/biometria_final.csv')
le = LabelEncoder()
y = le.fit_transform(df['cow_id'])

# Seleção de colunas puramente estruturais
cols_geo = [c for c in df.columns if c.startswith('geo_')]
# Filtramos apenas HOG e LBP, ignorando color_mean e sift
cols_img_struct = [c for c in df.columns if 'img_hog_' in c or 'img_lbp_' in c]

X_geometria = df[cols_geo]
X_hibrido_struct = df[cols_geo + cols_img_struct]

print(f"📊 Geometria: {X_geometria.shape[1]} colunas")
print(f"📊 Híbrido (Geo + HOG + LBP): {X_hibrido_struct.shape[1]} colunas")

📊 Geometria: 35 colunas
📊 Híbrido (Geo + HOG + LBP): 1809 colunas


In [3]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

params_mlp = {
    'classifier__hidden_layer_sizes': [(512, 256), (1024, 512), (512, 512, 256)],
    'classifier__activation': ['relu', 'tanh'],
    'classifier__alpha': [0.0001, 0.001, 0.01],
    'classifier__learning_rate': ['adaptive'],
    'classifier__solver': ['adam']
}

def executar_treino(X, y, titulo):
    print(f"\n{'='*20} {titulo} {'='*20}")
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', MLPClassifier(max_iter=1500, random_state=42))
    ])
    grid = GridSearchCV(pipe, params_mlp, cv=skf, scoring='accuracy', n_jobs=-1, verbose=2)
    grid.fit(X, y)
    print(f"🏆 Melhor Acurácia: {grid.best_score_:.2%}")
    print(f"📌 Melhor Setup: {grid.best_params_}")

In [4]:
executar_treino(X_geometria, y, "CENÁRIO 1: Geometria Pura (Base)")
executar_treino(X_hibrido_struct, y, "CENÁRIO 2: Independente de Cor (Geo + HOG + LBP)")


==================== CENÁRIO 1: Geometria Pura (Base) ====================
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END classifier__activation=relu, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(1024, 512), classifier__learning_rate=adaptive, classifier__solver=adam; total time= 5.5min
[CV] END classifier__activation=relu, classifier__alpha=0.01, classifier__hidden_layer_sizes=(512, 256), classifier__learning_rate=adaptive, classifier__solver=adam; total time= 2.2min
[CV] END classifier__activation=relu, classifier__alpha=0.01, classifier__hidden_layer_sizes=(512, 512, 256), classifier__learning_rate=adaptive, classifier__solver=adam; total time= 3.2min
[CV] END classifier__activation=tanh, classifier__alpha=0.0001, classifier__hidden_layer_sizes=(1024, 512), classifier__learning_rate=adaptive, classifier__solver=adam; total time=11.3min
[CV] END classifier__activation=tanh, classifier__alpha=0.01, classifier__hidden_layer_sizes=(512, 256), classif